# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_a
　　セッション1のAさんのスケーリングデータ　→　data1_a_label

In [2]:
%time
#sensor_dataの読み込み

data1_a = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_a = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

data1_a_label = pd.read_csv('emotion_label/A/emotion_A_20201111_1_50.csv')
data1_b_label = pd.read_csv('emotion_label/A/emotion_A_20201111_1_60.csv')
data1_c_label = pd.read_csv('emotion_label/A/emotion_A_20201111_1_70.csv')
data1_d_label = pd.read_csv('emotion_label/A/emotion_A_20201111_1_80.csv')
# data2_a_label = pd.read_csv('emotion_label/label/emotion_A_20201111_2.csv')
# data2_b_label = pd.read_csv('emotion_label/label/emotion_B_20201111_2.csv')
# data2_c_label = pd.read_csv('emotion_label/label/emotion_C_20201111_2.csv')
# data2_d_label = pd.read_csv('emotion_label/label/emotion_D_20201111_2.csv')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.34 µs


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data1_d_label

,start_time,finish_time,emotion_level,Unnamed: 3,Unnamed: 4
0,5:48,5:52,1,NaN,NaN
1,5:49,5:53,1,NaN,NaN
2,5:50,5:54,1,NaN,NaN
3,5:51,5:55,1,NaN,NaN
4,5:52,5:56,1,NaN,NaN
...,...,...,...,...,...
632,20:19,20:23,2,NaN,NaN
633,20:20,20:24,2,NaN,NaN
634,20:21,20:25,2,NaN,NaN
635,20:22,20:26,2,NaN,NaN


In [4]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

data1_a_label = dropna(data1_a_label)
data1_b_label = dropna(data1_b_label)
data1_c_label = dropna(data1_c_label)
data1_d_label = dropna(data1_d_label)

In [5]:
data1_d_label

,start_time,finish_time,emotion_level
0,5:48,5:52,1
1,5:49,5:53,1
2,5:50,5:54,1
3,5:51,5:55,1
4,5:52,5:56,1
...,...,...,...
632,20:19,20:23,2
633,20:20,20:24,2
634,20:21,20:25,2
635,20:22,20:26,2


In [6]:
#生データの確認
display(data1_a)
display(data1_a_label)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,12:19:53.685,9428,483,4399,163,494,364
1,ags,12:19:53.690,9435,463,4382,316,391,407
2,ags,12:19:53.695,9449,451,4291,322,263,377
3,ags,12:19:53.700,9467,453,4174,181,439,328
4,ags,12:19:53.705,9481,309,3989,-25,232,352
...,...,...,...,...,...,...,...,...
461018,ags,12:58:19.040,9012,-1399,3879,614,994,-75
461019,ags,12:58:19.045,8873,-1286,3920,578,854,-318
461020,ags,12:58:19.050,9110,-1438,3718,584,787,-373
461021,ags,12:58:19.055,9279,-1206,3920,517,726,-519


,start_time,finish_time,emotion_level
0,5:48,5:52,1
1,5:49,5:53,1
2,5:50,5:54,1
3,5:51,5:55,1
4,5:52,5:56,1
...,...,...,...
682,20:19,20:23,2
683,20:20,20:24,2
684,20:21,20:25,2
685,20:22,20:26,2


In [7]:
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data1_a)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
# norm_order(data2_a)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [8]:
data1_a

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,12:19:53.685,9428,483,4399,163,494,364,10414.973548,634.902355
1,ags,12:19:53.690,9435,463,4382,316,391,407,10413.237633,646.827643
2,ags,12:19:53.695,9449,451,4291,322,263,377,10387.477220,561.232572
3,ags,12:19:53.700,9467,453,4174,181,439,328,10356.233582,577.118705
4,ags,12:19:53.705,9481,309,3989,-25,232,352,10290.625005,422.318600
...,...,...,...,...,...,...,...,...,...,...
461018,ags,12:58:19.040,9012,-1399,3879,614,994,-75,9910.599679,1170.750614
461019,ags,12:58:19.045,8873,-1286,3920,578,854,-318,9785.209502,1079.131132
461020,ags,12:58:19.050,9110,-1438,3718,584,787,-373,9944.016693,1048.596204
461021,ags,12:58:19.055,9279,-1206,3920,517,726,-519,10144.982849,1031.370932


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [9]:
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data1_a)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

# datetime_order(data2_a)
# datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [10]:
data1_a_label

,start_time,finish_time,emotion_level
0,5:48,5:52,1
1,5:49,5:53,1
2,5:50,5:54,1
3,5:51,5:55,1
4,5:52,5:56,1
...,...,...,...
682,20:19,20:23,2
683,20:20,20:24,2
684,20:21,20:25,2
685,20:22,20:26,2


In [11]:
data1_a_label['start_time'] = pd.to_datetime(data1_a_label['start_time'], format='%M:%S')
data1_a_label['finish_time'] = pd.to_datetime(data1_a_label['finish_time'], format='%M:%S')

data1_b_label['start_time'] = pd.to_datetime(data1_b_label['start_time'], format='%M:%S')
data1_b_label['finish_time'] = pd.to_datetime(data1_b_label['finish_time'], format='%M:%S')

data1_c_label['start_time'] = pd.to_datetime(data1_c_label['start_time'], format='%M:%S')
data1_c_label['finish_time'] = pd.to_datetime(data1_c_label['finish_time'], format='%M:%S')

data1_d_label['start_time'] = pd.to_datetime(data1_d_label['start_time'], format='%M:%S')
data1_d_label['finish_time'] = pd.to_datetime(data1_d_label['finish_time'], format='%M:%S')

<ipython-input-11-7735d020c143>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_d_label['start_time'] = pd.to_datetime(data1_d_label['start_time'], format='%M:%S')
<ipython-input-11-7735d020c143>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_d_label['finish_time'] = pd.to_datetime(data1_d_label['finish_time'], format='%M:%S')


In [12]:
#emotion_labelのtimeHMS列をdatetime型に変換
# data1_a_label['timeHMS'] = pd.to_datetime(data1_a_label['timeHMS'], format='%M:%S')
# data1_b_label['timeHMS'] = pd.to_datetime(data1_b_label['timeHMS'], format='%M:%S')
# data1_c_label['timeHMS'] = pd.to_datetime(data1_c_label['timeHMS'], format='%M:%S')
# data1_d_label['timeHMS'] = pd.to_datetime(data1_d_label['timeHMS'], format='%H:%M:%S')

# data2_a_label['timeHMS'] = pd.to_datetime(data2_a_label['timeHMS'], format='%H:%M:%S')
# data2_b_label['timeHMS'] = pd.to_datetime(data2_b_label['timeHMS'], format='%M:%S')
# data2_c_label['timeHMS'] = pd.to_datetime(data2_c_label['timeHMS'], format='%H:%M:%S')
# data2_d_label['timeHMS'] = pd.to_datetime(data2_d_label['timeHMS'], format='%H:%M:%S')

In [13]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a)
display(data1_a_label)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:19:53.685,9428,483,4399,163,494,364,10414.973548,634.902355
1,ags,2020-11-11 12:19:53.690,9435,463,4382,316,391,407,10413.237633,646.827643
2,ags,2020-11-11 12:19:53.695,9449,451,4291,322,263,377,10387.477220,561.232572
3,ags,2020-11-11 12:19:53.700,9467,453,4174,181,439,328,10356.233582,577.118705
4,ags,2020-11-11 12:19:53.705,9481,309,3989,-25,232,352,10290.625005,422.318600
...,...,...,...,...,...,...,...,...,...,...
461018,ags,2020-11-11 12:58:19.040,9012,-1399,3879,614,994,-75,9910.599679,1170.750614
461019,ags,2020-11-11 12:58:19.045,8873,-1286,3920,578,854,-318,9785.209502,1079.131132
461020,ags,2020-11-11 12:58:19.050,9110,-1438,3718,584,787,-373,9944.016693,1048.596204
461021,ags,2020-11-11 12:58:19.055,9279,-1206,3920,517,726,-519,10144.982849,1031.370932


,start_time,finish_time,emotion_level
0,1900-01-01 00:05:48,1900-01-01 00:05:52,1
1,1900-01-01 00:05:49,1900-01-01 00:05:53,1
2,1900-01-01 00:05:50,1900-01-01 00:05:54,1
3,1900-01-01 00:05:51,1900-01-01 00:05:55,1
4,1900-01-01 00:05:52,1900-01-01 00:05:56,1
...,...,...,...
682,1900-01-01 00:20:19,1900-01-01 00:20:23,2
683,1900-01-01 00:20:20,1900-01-01 00:20:24,2
684,1900-01-01 00:20:21,1900-01-01 00:20:25,2
685,1900-01-01 00:20:22,1900-01-01 00:20:26,2


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [14]:
data1_a_label['start_time'] = data1_a_label['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_a_label['finish_time'] = data1_a_label['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

data1_b_label['start_time'] = data1_b_label['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label['finish_time'] = data1_b_label['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

data1_c_label['start_time'] = data1_c_label['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label['finish_time'] = data1_c_label['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

data1_d_label['start_time'] = data1_d_label['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label['finish_time'] = data1_d_label['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

<ipython-input-14-b6f7503e8bdd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_d_label['start_time'] = data1_d_label['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
<ipython-input-14-b6f7503e8bdd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_d_label['finish_time'] = data1_d_label['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)


In [15]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_d_label)

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:18,2020-11-11 12:40:22,1
1,2020-11-11 12:40:19,2020-11-11 12:40:23,1
2,2020-11-11 12:40:20,2020-11-11 12:40:24,1
3,2020-11-11 12:40:21,2020-11-11 12:40:25,1
4,2020-11-11 12:40:22,2020-11-11 12:40:26,1
...,...,...,...
632,2020-11-11 12:54:49,2020-11-11 12:54:53,2
633,2020-11-11 12:54:50,2020-11-11 12:54:54,2
634,2020-11-11 12:54:51,2020-11-11 12:54:55,2
635,2020-11-11 12:54:52,2020-11-11 12:54:56,2


In [16]:
#datetime化したラベルデータを保存
# data1_a_label.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_b_label.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_c_label.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_d_label.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_a_label.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_b_label.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_c_label.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_d_label.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [17]:
#datetime化した加速度及び角速度データを保存
# data1_a.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_a.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [18]:
#時刻をインデックスに
data1_a.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

# data2_a.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [19]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 12:19:53.685,ags,9428,483,4399,163,494,364,10414.973548,634.902355
2020-11-11 12:19:53.690,ags,9435,463,4382,316,391,407,10413.237633,646.827643
2020-11-11 12:19:53.695,ags,9449,451,4291,322,263,377,10387.477220,561.232572
2020-11-11 12:19:53.700,ags,9467,453,4174,181,439,328,10356.233582,577.118705
2020-11-11 12:19:53.705,ags,9481,309,3989,-25,232,352,10290.625005,422.318600
...,...,...,...,...,...,...,...,...,...
2020-11-11 12:58:19.040,ags,9012,-1399,3879,614,994,-75,9910.599679,1170.750614
2020-11-11 12:58:19.045,ags,8873,-1286,3920,578,854,-318,9785.209502,1079.131132
2020-11-11 12:58:19.050,ags,9110,-1438,3718,584,787,-373,9944.016693,1048.596204


In [20]:
data1_a_label["start_time"] = data1_a_label["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_a_label["finish_time"] = data1_a_label["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

data1_b_label["start_time"] = data1_b_label["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label["finish_time"] = data1_b_label["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

data1_c_label["start_time"] = data1_c_label["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label["finish_time"] = data1_c_label["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

data1_d_label["start_time"] = data1_d_label["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label["finish_time"] = data1_d_label["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

<ipython-input-20-04283f48f4b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_d_label["start_time"] = data1_d_label["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
<ipython-input-20-04283f48f4b8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_d_label["finish_time"] = data1_d_label["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")


In [21]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_c_label)

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:18,2020-11-11 12:40:22,1
1,2020-11-11 12:40:19,2020-11-11 12:40:23,1
2,2020-11-11 12:40:20,2020-11-11 12:40:24,1
3,2020-11-11 12:40:21,2020-11-11 12:40:25,1
4,2020-11-11 12:40:22,2020-11-11 12:40:26,1
...,...,...,...
649,2020-11-11 12:54:49,2020-11-11 12:54:53,2
650,2020-11-11 12:54:50,2020-11-11 12:54:54,2
651,2020-11-11 12:54:51,2020-11-11 12:54:55,2
652,2020-11-11 12:54:52,2020-11-11 12:54:56,2


In [22]:
data1_a_start_time = list(data1_a_label["start_time"])
data1_a_finish_time = list(data1_a_label["finish_time"])

data1_b_start_time = list(data1_b_label["start_time"])
data1_b_finish_time = list(data1_b_label["finish_time"])

data1_c_start_time = list(data1_c_label["start_time"])
data1_c_finish_time = list(data1_c_label["finish_time"])

data1_d_start_time = list(data1_d_label["start_time"])
data1_d_finish_time = list(data1_d_label["finish_time"])

In [23]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

data1_a_record_1 = sensor_segmentation_order(data1_a, data1_a_start_time, data1_a_finish_time, data1_a_label)
data1_a_record_2 = sensor_segmentation_order(data1_a, data1_b_start_time, data1_b_finish_time, data1_b_label)
data1_a_record_3 = sensor_segmentation_order(data1_a, data1_c_start_time, data1_c_finish_time, data1_c_label)
data1_a_record_4 = sensor_segmentation_order(data1_a, data1_d_start_time, data1_d_finish_time, data1_d_label)

In [24]:
len(data1_a_record_4)

637

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [25]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

data1_a_record_mean = acc_gyr_mean_order(data1_a_record_1, data1_a_label)
data1_b_record_mean = acc_gyr_mean_order(data1_a_record_2, data1_b_label)
data1_c_record_mean = acc_gyr_mean_order(data1_a_record_3, data1_c_label)
data1_d_record_mean = acc_gyr_mean_order(data1_a_record_4, data1_d_label)

# data2_a_record_mean = acc_gyr_mean_order(data2_a_record, data2_a_label)
# data2_b_record_mean = acc_gyr_mean_order(data2_b_record, data2_b_label)
# data2_c_record_mean = acc_gyr_mean_order(data2_c_record, data2_c_label)
# data2_d_record_mean = acc_gyr_mean_order(data2_d_record, data2_d_label)

# a = data1_a_record_mean
# b = data1_b_record_mean
# c = data1_c_record_mean
# d = data1_d_record_mean

# e = data2_a_record_mean
# f = data2_b_record_mean
# g = data2_c_record_mean
# h = data2_d_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [26]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

data1_a_record_var = acc_gyr_var_order(data1_a_record_1, data1_a_label)
data1_b_record_var = acc_gyr_var_order(data1_a_record_2, data1_b_label)
data1_c_record_var = acc_gyr_var_order(data1_a_record_3, data1_c_label)
data1_d_record_var = acc_gyr_var_order(data1_a_record_4, data1_d_label)

# data2_a_record_var = acc_gyr_var_order(data2_a_record, data2_a_label)
# data2_b_record_var = acc_gyr_var_order(data2_b_record, data2_b_label)
# data2_c_record_var = acc_gyr_var_order(data2_c_record, data2_c_label)
# data2_d_record_var = acc_gyr_var_order(data2_d_record, data2_d_label)

# a = data1_a_record_var
# b = data1_b_record_var
# c = data1_c_record_var
# d = data1_d_record_var

# e = data2_a_record_var
# f = data2_b_record_var
# g = data2_c_record_var
# h = data2_d_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [27]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

data1_a_record_max = acc_gyr_max_order(data1_a_record_1, data1_a_label)
data1_b_record_max = acc_gyr_max_order(data1_a_record_2, data1_b_label)
data1_c_record_max = acc_gyr_max_order(data1_a_record_3, data1_c_label)
data1_d_record_max = acc_gyr_max_order(data1_a_record_4, data1_d_label)

# data2_a_record_max = acc_gyr_max_order(data2_a_record, data2_a_label)
# data2_b_record_max = acc_gyr_max_order(data2_b_record, data2_b_label)
# data2_c_record_max = acc_gyr_max_order(data2_c_record, data2_c_label)
# data2_d_record_max = acc_gyr_max_order(data2_d_record, data2_d_label)

# a = data1_a_record_max
# b = data1_b_record_max
# c = data1_c_record_max
# d = data1_d_record_max

# e = data2_a_record_max
# f = data2_b_record_max
# g = data2_c_record_max
# h = data2_d_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [28]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

data1_a_record_min = acc_gyr_min_order(data1_a_record_1, data1_a_label)
data1_b_record_min = acc_gyr_min_order(data1_a_record_2, data1_b_label)
data1_c_record_min = acc_gyr_min_order(data1_a_record_3, data1_c_label)
data1_d_record_min = acc_gyr_min_order(data1_a_record_4, data1_d_label)

# data2_a_record_min = acc_gyr_min_order(data2_a_record, data2_a_label)
# data2_b_record_min = acc_gyr_min_order(data2_b_record, data2_b_label)
# data2_c_record_min = acc_gyr_min_order(data2_c_record, data2_c_label)
# data2_d_record_min = acc_gyr_min_order(data2_d_record, data2_d_label)

# a = data1_a_record_min
# b = data1_b_record_min
# c = data1_c_record_min
# d = data1_d_record_min

# e = data2_a_record_min
# f = data2_b_record_min
# g = data2_c_record_min
# h = data2_d_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [29]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

data1_a_record_fq_amp_max = fq_amp_max_order(data1_a_record_1, data1_a_label)
data1_b_record_fq_amp_max = fq_amp_max_order(data1_a_record_2, data1_b_label)
data1_c_record_fq_amp_max = fq_amp_max_order(data1_a_record_3, data1_c_label)
data1_d_record_fq_amp_max = fq_amp_max_order(data1_a_record_4, data1_d_label)

# data2_a_record_fq_amp_max = fq_amp_max_order(data2_a_record, data2_a_label)
# data2_b_record_fq_amp_max = fq_amp_max_order(data2_b_record, data2_b_label)
# data2_c_record_fq_amp_max = fq_amp_max_order(data2_c_record, data2_c_label)
# data2_d_record_fq_amp_max = fq_amp_max_order(data2_d_record, data2_d_label)

# a = data1_a_record_fq_amp_max
# b = data1_b_record_fq_amp_max
# c = data1_c_record_fq_amp_max
# d = data1_d_record_fq_amp_max

# e = data2_a_record_fq_amp_max
# f = data2_b_record_fq_amp_max
# g = data2_c_record_fq_amp_max
# h = data2_d_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [30]:
#全特徴量の行列の導出
data1_a_record_all_feature = pd.concat([data1_a_record_mean, data1_a_record_var, data1_a_record_max, data1_a_record_min, data1_a_record_fq_amp_max], axis=1, ignore_index=False)
data1_b_record_all_feature = pd.concat([data1_b_record_mean, data1_b_record_var, data1_b_record_max, data1_b_record_min, data1_b_record_fq_amp_max], axis=1, ignore_index=False)
data1_c_record_all_feature = pd.concat([data1_c_record_mean, data1_c_record_var, data1_c_record_max, data1_c_record_min, data1_c_record_fq_amp_max], axis=1, ignore_index=False)
data1_d_record_all_feature = pd.concat([data1_d_record_mean, data1_d_record_var, data1_d_record_max, data1_d_record_min, data1_d_record_fq_amp_max], axis=1, ignore_index=False)

# data2_a_record_all_feature = pd.concat([data2_a_record_mean, data2_a_record_var, data2_a_record_max, data2_a_record_min, data2_a_record_fq_amp_max], axis=1, ignore_index=False)
# data2_b_record_all_feature = pd.concat([data2_b_record_mean, data2_b_record_var, data2_b_record_max, data2_b_record_min, data2_b_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_record_all_feature = pd.concat([data2_c_record_mean, data2_c_record_var, data2_c_record_max, data2_c_record_min, data2_c_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_record_all_feature = pd.concat([data2_d_record_mean, data2_d_record_var, data2_d_record_max, data2_d_record_min, data2_d_record_fq_amp_max], axis=1, ignore_index=False)

In [31]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_b_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,9989.120,419.244,-2052.121,22.665,5.644,5.692,46958.269600,2.519337e+04,1.210483e+05,205324.582775,...,9205,-20,-3095,-1501,-2919,-562,6.653620,90.358921,2.739726,327.603159
1,9988.680,374.708,-2093.075,10.716,40.419,-23.403,46205.525600,2.660551e+04,9.165247e+04,224497.813344,...,9205,-437,-3095,-1501,-2919,-714,5.870841,81.049321,1.174168,324.534956
2,10000.110,225.928,-2044.056,210.418,210.142,167.292,55977.995900,1.996043e+05,1.674371e+05,440862.123276,...,9012,-1797,-3188,-1501,-2468,-714,5.479452,169.189386,0.391389,421.393224
3,10008.944,-301.132,-1579.636,259.055,385.835,304.251,51479.100864,1.098852e+06,8.099625e+05,405569.557975,...,9012,-2847,-3188,-1501,-2468,-714,5.479452,141.644558,0.391389,562.283975
4,9981.447,-926.682,-977.567,231.636,435.985,297.311,56865.437191,1.851899e+06,1.622998e+06,417799.129504,...,9012,-3279,-3188,-2049,-2212,-1074,5.479452,124.008126,0.391389,629.463949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,9832.002,376.364,-2677.213,111.254,73.332,-87.704,30536.553996,3.426154e+04,1.875881e+05,91214.675484,...,9200,-47,-3789,-933,-2060,-812,8.610568,72.262946,0.782779,378.511950
682,9844.242,441.944,-2593.380,37.436,108.618,15.572,61678.375436,6.564020e+04,2.587700e+05,388514.269904,...,8739,-1098,-3789,-4580,-2108,-812,5.870841,52.258742,0.782779,392.086007
683,9820.866,323.123,-2669.401,-241.840,-48.918,71.333,78599.508044,1.586836e+05,4.554547e+05,591048.760400,...,8739,-1098,-4599,-4580,-2657,-1037,9.393346,46.718720,0.391389,366.460464
684,9706.397,324.454,-2956.586,-242.307,-201.340,50.513,146288.667391,1.591388e+05,9.517278e+05,566086.768751,...,8739,-1098,-4812,-4580,-2657,-1037,15.655577,36.202566,0.391389,155.225788


In [32]:
#各特徴量データの保存
data1_a_record_all_feature.to_csv('feature/A/data1_a_50_record_all_feature.csv', index=False)
data1_b_record_all_feature.to_csv('feature/A/data1_a_60_record_all_feature.csv', index=False)
data1_c_record_all_feature.to_csv('feature/A/data1_a_70_record_all_feature.csv', index=False)
data1_d_record_all_feature.to_csv('feature/A/data1_a_80_record_all_feature.csv', index=False)

# data2_a_record_all_feature.to_csv('feature/feature_record/data2_a_record_all_feature_kai.csv', index=False)
# data2_b_record_all_feature.to_csv('feature/feature_record/data2_b_record_all_feature_kai.csv', index=False)
# data2_c_record_all_feature.to_csv('feature/feature_record/data2_c_record_all_feature_kai.csv', index=False)
# data2_d_record_all_feature.to_csv('feature/feature_record/data2_d_record_all_feature_kai.csv', index=False)

In [33]:
data1_a_label

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:18,2020-11-11 12:40:22,1
1,2020-11-11 12:40:19,2020-11-11 12:40:23,1
2,2020-11-11 12:40:20,2020-11-11 12:40:24,1
3,2020-11-11 12:40:21,2020-11-11 12:40:25,1
4,2020-11-11 12:40:22,2020-11-11 12:40:26,1
...,...,...,...
682,2020-11-11 12:54:49,2020-11-11 12:54:53,2
683,2020-11-11 12:54:50,2020-11-11 12:54:54,2
684,2020-11-11 12:54:51,2020-11-11 12:54:55,2
685,2020-11-11 12:54:52,2020-11-11 12:54:56,2


In [34]:
data1_a_label_time = data1_a_label.drop("emotion_level", axis=1)
data1_b_label_time = data1_b_label.drop("emotion_level", axis=1)
data1_c_label_time = data1_c_label.drop("emotion_level", axis=1)
data1_d_label_time = data1_d_label.drop("emotion_level", axis=1)

In [35]:
data1_a_label_time

,start_time,finish_time
0,2020-11-11 12:40:18,2020-11-11 12:40:22
1,2020-11-11 12:40:19,2020-11-11 12:40:23
2,2020-11-11 12:40:20,2020-11-11 12:40:24
3,2020-11-11 12:40:21,2020-11-11 12:40:25
4,2020-11-11 12:40:22,2020-11-11 12:40:26
...,...,...
682,2020-11-11 12:54:49,2020-11-11 12:54:53
683,2020-11-11 12:54:50,2020-11-11 12:54:54
684,2020-11-11 12:54:51,2020-11-11 12:54:55
685,2020-11-11 12:54:52,2020-11-11 12:54:56


In [36]:
data1_b_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,9989.120,419.244,-2052.121,22.665,5.644,5.692,46958.269600,2.519337e+04,1.210483e+05,205324.582775,...,9205,-20,-3095,-1501,-2919,-562,6.653620,90.358921,2.739726,327.603159
1,9988.680,374.708,-2093.075,10.716,40.419,-23.403,46205.525600,2.660551e+04,9.165247e+04,224497.813344,...,9205,-437,-3095,-1501,-2919,-714,5.870841,81.049321,1.174168,324.534956
2,10000.110,225.928,-2044.056,210.418,210.142,167.292,55977.995900,1.996043e+05,1.674371e+05,440862.123276,...,9012,-1797,-3188,-1501,-2468,-714,5.479452,169.189386,0.391389,421.393224
3,10008.944,-301.132,-1579.636,259.055,385.835,304.251,51479.100864,1.098852e+06,8.099625e+05,405569.557975,...,9012,-2847,-3188,-1501,-2468,-714,5.479452,141.644558,0.391389,562.283975
4,9981.447,-926.682,-977.567,231.636,435.985,297.311,56865.437191,1.851899e+06,1.622998e+06,417799.129504,...,9012,-3279,-3188,-2049,-2212,-1074,5.479452,124.008126,0.391389,629.463949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,9832.002,376.364,-2677.213,111.254,73.332,-87.704,30536.553996,3.426154e+04,1.875881e+05,91214.675484,...,9200,-47,-3789,-933,-2060,-812,8.610568,72.262946,0.782779,378.511950
682,9844.242,441.944,-2593.380,37.436,108.618,15.572,61678.375436,6.564020e+04,2.587700e+05,388514.269904,...,8739,-1098,-3789,-4580,-2108,-812,5.870841,52.258742,0.782779,392.086007
683,9820.866,323.123,-2669.401,-241.840,-48.918,71.333,78599.508044,1.586836e+05,4.554547e+05,591048.760400,...,8739,-1098,-4599,-4580,-2657,-1037,9.393346,46.718720,0.391389,366.460464
684,9706.397,324.454,-2956.586,-242.307,-201.340,50.513,146288.667391,1.591388e+05,9.517278e+05,566086.768751,...,8739,-1098,-4812,-4580,-2657,-1037,15.655577,36.202566,0.391389,155.225788


In [37]:
data1_a_record_all_feature = pd.concat([data1_a_label_time, data1_a_record_all_feature], axis=1)
data1_b_record_all_feature = pd.concat([data1_b_label_time, data1_b_record_all_feature], axis=1)
data1_c_record_all_feature = pd.concat([data1_c_label_time, data1_c_record_all_feature], axis=1)
data1_d_record_all_feature = pd.concat([data1_d_label_time, data1_d_record_all_feature], axis=1)

In [38]:
data1_a_record_all_feature.to_csv('feature/A/data1_a_50_record_all_feature.csv', index=False)
data1_b_record_all_feature.to_csv('feature/A/data1_a_60_record_all_feature.csv', index=False)
data1_c_record_all_feature.to_csv('feature/A/data1_a_70_record_all_feature.csv', index=False)
data1_d_record_all_feature.to_csv('feature/A/data1_a_80_record_all_feature.csv', index=False)

In [39]:
data1_b_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:40:18,2020-11-11 12:40:22,9989.120,419.244,-2052.121,22.665,5.644,5.692,46958.269600,2.519337e+04,...,9205,-20,-3095,-1501,-2919,-562,6.653620,90.358921,2.739726,327.603159
1,2020-11-11 12:40:19,2020-11-11 12:40:23,9988.680,374.708,-2093.075,10.716,40.419,-23.403,46205.525600,2.660551e+04,...,9205,-437,-3095,-1501,-2919,-714,5.870841,81.049321,1.174168,324.534956
2,2020-11-11 12:40:20,2020-11-11 12:40:24,10000.110,225.928,-2044.056,210.418,210.142,167.292,55977.995900,1.996043e+05,...,9012,-1797,-3188,-1501,-2468,-714,5.479452,169.189386,0.391389,421.393224
3,2020-11-11 12:40:21,2020-11-11 12:40:25,10008.944,-301.132,-1579.636,259.055,385.835,304.251,51479.100864,1.098852e+06,...,9012,-2847,-3188,-1501,-2468,-714,5.479452,141.644558,0.391389,562.283975
4,2020-11-11 12:40:22,2020-11-11 12:40:26,9981.447,-926.682,-977.567,231.636,435.985,297.311,56865.437191,1.851899e+06,...,9012,-3279,-3188,-2049,-2212,-1074,5.479452,124.008126,0.391389,629.463949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,2020-11-11 12:54:49,2020-11-11 12:54:53,9832.002,376.364,-2677.213,111.254,73.332,-87.704,30536.553996,3.426154e+04,...,9200,-47,-3789,-933,-2060,-812,8.610568,72.262946,0.782779,378.511950
682,2020-11-11 12:54:50,2020-11-11 12:54:54,9844.242,441.944,-2593.380,37.436,108.618,15.572,61678.375436,6.564020e+04,...,8739,-1098,-3789,-4580,-2108,-812,5.870841,52.258742,0.782779,392.086007
683,2020-11-11 12:54:51,2020-11-11 12:54:55,9820.866,323.123,-2669.401,-241.840,-48.918,71.333,78599.508044,1.586836e+05,...,8739,-1098,-4599,-4580,-2657,-1037,9.393346,46.718720,0.391389,366.460464
684,2020-11-11 12:54:52,2020-11-11 12:54:56,9706.397,324.454,-2956.586,-242.307,-201.340,50.513,146288.667391,1.591388e+05,...,8739,-1098,-4812,-4580,-2657,-1037,15.655577,36.202566,0.391389,155.225788
